In [3]:
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time

In [4]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [5]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, 
                                                                               trainset.target, 
                                                                               ONEHOT, test_size = 0.2)

In [6]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [691, 563, 45, 32, 1785, 7006, 3661, 657, 65, 1006] ['flawed', 'worthy', 'look', 'life', 'u', 'relocation', 'camps', 'words', 'another', 'sports']


In [7]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [8]:
class Model:
    def __init__(self, embedded_size, dict_size, dimension_output, learning_rate,
                top_k=5, n_filters=250):
        kernels = [3, 4, 5]
        
        def add_conv1d(x, n_filters, kernel_size, strides=1):
            return tf.layers.conv1d(inputs = x,
                            filters = n_filters,
                            kernel_size  = kernel_size,
                            strides = strides,
                            padding = 'valid',
                            use_bias = True,
                            activation = tf.nn.relu)
        
        def add_kmax_pooling(x):
            Y = tf.transpose(x, [0, 2, 1])
            Y = tf.nn.top_k(Y, top_k, sorted=False).values
            Y = tf.transpose(Y, [0, 2, 1])
            return tf.reshape(Y, [-1, top_k, n_filters//len(kernels)])
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        parallels = []
        for k in kernels:
            p = add_conv1d(encoder_embedded, n_filters//len(kernels), kernel_size=k)
            p = add_kmax_pooling(p)
            parallels.append(p)
            
        parallels = tf.concat(parallels, axis=-1)
        parallels = tf.reshape(parallels, [-1, top_k * (len(kernels)*(n_filters//len(kernels)))])
        feed = tf.layers.dense(parallels, n_filters, tf.nn.relu)
        self.logits = tf.layers.dense(parallels, dimension_output)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(embedded_size,vocabulary_size+4,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : test_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.584473
time taken: 5.887679576873779
epoch: 0, training loss: 0.725877, training acc: 0.548532, valid loss: 0.671398, valid acc: 0.584473

epoch: 1, pass acc: 0.584473, current acc: 0.617188
time taken: 4.146975040435791
epoch: 1, training loss: 0.493067, training acc: 0.814631, valid loss: 0.656880, valid acc: 0.617188

epoch: 2, pass acc: 0.617188, current acc: 0.633301
time taken: 4.16161036491394
epoch: 2, training loss: 0.352320, training acc: 0.916785, valid loss: 0.649146, valid acc: 0.633301

epoch: 3, pass acc: 0.633301, current acc: 0.645020
time taken: 4.176854848861694
epoch: 3, training loss: 0.238340, training acc: 0.960464, valid loss: 0.650460, valid acc: 0.645020

epoch: 4, pass acc: 0.645020, current acc: 0.657715
time taken: 4.182652711868286
epoch: 4, training loss: 0.163666, training acc: 0.976326, valid loss: 0.675792, valid acc: 0.657715

time taken: 4.179717540740967
epoch: 5, training loss: 0.123527, training acc: 0.

In [12]:
logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_X,dictionary,maxlen)})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

              precision    recall  f1-score   support

    negative       0.71      0.70      0.70      1084
    positive       0.69      0.70      0.70      1049

    accuracy                           0.70      2133
   macro avg       0.70      0.70      0.70      2133
weighted avg       0.70      0.70      0.70      2133

